In [0]:
%sql
create schema processing_catalog.schema_harmonized_facts

In [0]:
%sql
CREATE TABLE IF NOT EXISTS processing_catalog.schema_harmonized_facts.t_sales_transactions (
    invoice_id STRING NOT NULL COMMENT 'Primary key: Unique identifier for the transaction',
    transaction_date_time TIMESTAMP NOT NULL COMMENT 'Timestamp when the transaction occurred',
    store_id STRING NOT NULL COMMENT 'Identifier for the store where the transaction occurred',
    customer_id STRING COMMENT 'Identifier for the customer involved in the transaction',
    employee_id STRING COMMENT 'Identifier for the employee handling the transaction',
    payment_type STRING COMMENT 'Type of payment used in the transaction',
    total_amount DECIMAL(10, 2) COMMENT 'Total amount for the transaction',
    -- AUDIT columns
    batch_r_timestamp TIMESTAMP COMMENT 'Audit column showing when the corresponding raw record was inserted',
    h_insert_timestamp TIMESTAMP DEFAULT current_timestamp() COMMENT 'Audit column showing when the record was inserted',
    h_update_timestamp TIMESTAMP DEFAULT current_timestamp() COMMENT 'Audit column showing when the record was updated',
    job_id STRING NOT NULL COMMENT 'Audit column showing the job that inserted/updated the record',
    -- PARTITION column
    transaction_month DATE GENERATED ALWAYS AS (CAST(DATE_TRUNC('MONTH', transaction_date_time) AS DATE)) COMMENT 'Partition column',
    CONSTRAINT t_sales_transactions_pkey PRIMARY KEY (invoice_id)
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')
COMMENT 'Harmonized table for sales transactions data'
PARTITIONED BY (transaction_month);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS processing_catalog.schema_harmonized_facts.t_sales_line_items (
    line_item_id STRING NOT NULL COMMENT 'Primary key: Unique identifier for the line item',
    invoice_id STRING NOT NULL COMMENT 'Foreign key: Identifier for the associated transaction',
    product_id STRING NOT NULL COMMENT 'Identifier for the product sold',
    quantity INT COMMENT 'Quantity of the product sold',
    unit_price DECIMAL(10, 2) COMMENT 'Price per unit of the product',
    total_price DECIMAL(10, 2) COMMENT 'Total price for the line item',
    discount_applied DECIMAL(10, 2) COMMENT 'Discount applied to the line item',
    -- AUDIT columns
    batch_r_timestamp TIMESTAMP COMMENT 'Audit column showing when the corresponding raw record was inserted',
    h_insert_timestamp TIMESTAMP DEFAULT current_timestamp() COMMENT 'Audit column showing when the record was inserted',
    h_update_timestamp TIMESTAMP DEFAULT current_timestamp() COMMENT 'Audit column showing when the record was updated',
    job_id STRING NOT NULL COMMENT 'Audit column showing the job that inserted/updated the record',
    CONSTRAINT t_sales_line_items_pkey PRIMARY KEY (line_item_id)
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')
COMMENT 'Harmonized table for sales line items data';